In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline
import seaborn as sns 
from collections import Counter 
import warnings 
import math 
import itertools
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import top_k_accuracy_score
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('C:/Users/manul/Desktop/Major project/Project/DelhiNCR Restaurants.csv',encoding='latin-1')
df.columns
df.head

<bound method NDFrame.head of              Restaurant_Name  \
0                   Rustom's   
1                  Cafe Lota   
2     Dum-Pukht - ITC Maurya   
3                Burma Burma   
4              The Big Chill   
...                      ...   
1960          Bablu Chic-Inn   
1961               Kovilakam   
1962        Krips Restaurant   
1963                Cafe 147   
1964                Pizzasia   

                                               Category  Pricing_for_2  \
0     Parsi, Street Food, North Indian, Desserts, Be...           2100   
1           Cafe, South Indian, North Indian, Beverages           1200   
2                       Mughlai, North Indian, Desserts           5000   
3           Asian, Burmese, Bubble Tea, Desserts, Salad           1600   
4                        Continental, American, Italian           1500   
...                                                 ...            ...   
1960          Mughlai, North Indian, Chinese, Fast Food            

In [3]:
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1965 entries, 0 to 1964
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Restaurant_Name        1965 non-null   object 
 1   Category               1965 non-null   object 
 2   Pricing_for_2          1965 non-null   int64  
 3   Locality               1965 non-null   object 
 4   Dining_Rating          1965 non-null   float64
 5   Dining_Review_Count    1965 non-null   int64  
 6   Delivery_Rating        1563 non-null   float64
 7   Delivery_Rating_Count  1965 non-null   int64  
 8   Website                1965 non-null   object 
 9   Address                1965 non-null   object 
 10  Phone_No               1965 non-null   object 
 11  Latitude               1965 non-null   float64
 12  Longitude              1965 non-null   float64
 13  Known_For2             1560 non-null   object 
 14  Known_For22            1124 non-null   object 
dtypes: f

,Pricing_for_2,Dining_Rating,Dining_Review_Count,Delivery_Rating,Delivery_Rating_Count,Latitude,Longitude
count,1965.000000,1965.000000,1965.000000,1563.000000,1965.000000,1965.000000,1965.000000
mean,1090.687023,4.140000,721.954198,3.964299,3029.550636,28.549542,77.161570
std,934.481617,0.202147,932.618839,0.242305,6529.839896,0.784241,1.685399
min,100.000000,3.900000,3.000000,2.800000,0.000000,-5.975200,2.650463
25%,500.000000,4.000000,244.000000,3.800000,34.000000,28.496564,77.100745
50%,700.000000,4.100000,433.000000,4.000000,604.000000,28.567882,77.201558
75%,1500.000000,4.300000,842.000000,4.100000,3316.000000,28.635845,77.258408
max,8000.000000,4.900000,10100.000000,4.500000,87700.000000,29.065189,77.536009


In [4]:
df.dropna(inplace=True)
df['id'] = df.index

In [5]:
df['Final_rating'] = df.apply(lambda x: x['Dining_Rating'] * math.log10(x['Dining_Review_Count']), axis=1)
df['Locality'].unique()

array([' Pragati Maidan, New Delhi',
       ' Cyber Hub, DLF Cyber City, Gurgaon', ' Lodhi Colony, New Delhi',
       ' Sector 15, Gurgaon', ' Connaught Place, New Delhi',
       ' Kailash Colony, New Delhi',
       ' Two Horizon Center, Golf Course Road, Gurgaon',
       ' Vijay Nagar, New Delhi', ' Hauz Khas Village, New Delhi',
       ' Pandara Road Market, New Delhi', ' Mehrauli, New Delhi',
       ' Anand Vihar, New Delhi',
       ' Pacific Mall, Tagore Garden, New Delhi',
       ' Select Citywalk Mall, Saket, New Delhi', ' Sushant Lok, Gurgaon',
       ' Majnu ka Tila, New Delhi',
       ' MGF Metropolis Mall, MG Road, Gurgaon', ' Sector 23, Gurgaon',
       ' Suncity Business Tower, Golf Course Road, Gurgaon',
       ' DLF Mall of India, Sector 18,  Noida, Noida',
       ' Unity One Mall, Janakpuri, New Delhi',
       ' Cross Point Mall, DLF Phase 4, Gurgaon', ' Sector 110, Noida',
       ' DLF Avenue, Saket, New Delhi',
       ' Greater Kailash 2 (GK2), New Delhi',
       ' San

In [6]:
df['City'] = df.apply(lambda x: x['Locality'].split(",") , axis=1)
df['City'] = df.apply(lambda x: x['City'][-1].strip() , axis=1)
y=list(set(df['City']))
y.sort()
print(y)

['Faridabad', 'Ghaziabad', 'Greater Noida', 'Gurgaon', 'New Delhi', 'Noida']


In [7]:
for (i, item) in enumerate(y, start=1):
    print(i, item, end='\t')
city=y[int(input("Enter the location number ")) -1]
city

1 Faridabad	2 Ghaziabad	3 Greater Noida	4 Gurgaon	5 New Delhi	6 Noida	Enter the location number 5


'New Delhi'

In [8]:
df.drop(['Delivery_Rating','Delivery_Rating_Count','Dining_Review_Count','Website','Phone_No','Longitude','Latitude','Known_For2','Known_For22'],axis=1,inplace=True)

In [9]:
df.columns

Index(['Restaurant_Name', 'Category', 'Pricing_for_2', 'Locality',
       'Dining_Rating', 'Address', 'id', 'Final_rating', 'City'],
      dtype='object')

In [10]:
tf=df[df.City==city].copy()
tf.columns
#tf.drop(['Restaurant_Name','Locality'],axis=1,inplace=True)

Index(['Restaurant_Name', 'Category', 'Pricing_for_2', 'Locality',
       'Dining_Rating', 'Address', 'id', 'Final_rating', 'City'],
      dtype='object')

In [11]:
set(tf['City'])

{'New Delhi'}

In [12]:
m=tf['Final_rating'].max()
tf['Final_rating'] = tf.apply(lambda x: (x['Final_rating']*5)/m,axis=1)

In [13]:
tf['Category'] = tf.apply(lambda x: x['Category'].split(","),axis=1)
tf['Category'] = tf.apply(lambda x: list(i.strip() for i in x['Category']),axis=1)
print(tf['Category'])

x = list(tf.get('Category'))
x = list(set(itertools.chain.from_iterable(x)))

1           [Cafe, South Indian, North Indian, Beverages]
5                                          [South Indian]
7                                   [Ice Cream, Desserts]
9       [South Indian, North Indian, Fast Food, Desser...
12      [Cafe, Continental, Italian, Fast Food, Orient...
                              ...                        
1944                        [Bakery, Fast Food, Desserts]
1951                       [North Indian, Mughlai, Rolls]
1952                              [North Indian, Chinese]
1953                              [Mughlai, North Indian]
1955                    [Chinese, Tibetan, Thai, Seafood]
Name: Category, Length: 554, dtype: object


In [14]:
for (i, item) in enumerate(x, start=1):
    if i%5==0:
        t='\n'
    else:
        t='\t'
    print(i, item ,end=t)
categories=[]
for i in range(5):
    categories.append(x[int(input('select 5 categoies'))-1])

1 Pizza	2 Raw Meats	3 Mediterranean	4 Seafood	5 South Indian
6 Korean	7 European	8 Bakery	9 Nepalese	10 French
11 Cafe	12 Kerala	13 Juices	14 Street Food	15 Singaporean
16 Odia	17 Spanish	18 Hyderabadi	19 Sandwich	20 Mexican
21 Thai	22 Afghan	23 Coffee	24 Fast Food	25 Burmese
26 Rolls	27 Lebanese	28 Naga	29 Beverages	30 American
31 Salad	32 Tea	33 Bengali	34 Turkish	35 Asian
36 Momos	37 Chettinad	38 Mithai	39 Sushi	40 North Eastern
41 Mughlai	42 BBQ	43 Healthy Food	44 Oriental	45 Bar Food
46 Japanese	47 Biryani	48 Ice Cream	49 Pasta	50 Awadhi
51 Middle Eastern	52 Arabian	53 Rajasthani	54 Italian	55 North Indian
56 Finger Food	57 Kebab	58 Andhra	59 Burger	60 Wraps
61 Vietnamese	62 Gujarati	63 Tibetan	64 Continental	65 Iranian
66 Tex-Mex	67 Panini	68 Bihari	69 Assamese	70 Tamil
71 Desserts	72 Chinese	select 5 categoies1
select 5 categoies2
select 5 categoies3
select 5 categoies4
select 5 categoies5


In [15]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return len(lst3)

In [16]:
tf["CategoryMatch"] = tf.apply(lambda x: intersection(x['Category'],categories),axis=1)

In [17]:
j=tf['Pricing_for_2'].max()
tf['Pricing_index']=tf.apply(lambda x: (x['Pricing_for_2']*5)/j, axis=1)

In [18]:
x = tf[['Dining_Rating','Final_rating','Pricing_index','CategoryMatch']]
y = tf['id']

In [19]:
p = int(input("Enter price for 2 in multiple of 100\t"))
x_test = pd.DataFrame({'Dining_Rating':[5],'Final_rating':[5],'Pricing_index':[(p*5)/j],'CategoryMatch':[5]})

Enter price for 2 in multiple of 100	1500


In [20]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(x,y)

distance , y_pred = knn.kneighbors(x_test,n_neighbors=5)
distance , y_pred

(array([[3.19974096, 3.31906286, 3.45658971, 3.63821145, 3.91679255]]),
 array([[ 14, 327, 158, 336,   7]], dtype=int64))

In [21]:
tf.iloc[y_pred[0]]

,Restaurant_Name,Category,Pricing_for_2,Locality,Dining_Rating,Address,id,Final_rating,City,CategoryMatch,Pricing_index
66,Mahabelly,"[Kerala, South Indian, Seafood, Biryani]",700,"DLF Avenue, Saket, New Delhi",4.6,"MA-A05, Ground Floor, Restaurant Block, DLF Av...",66,4.319416,New Delhi,2,0.686275
1179,Ardor 2.1,"[North Indian, Chinese, Continental, Italian, ...",1800,"Connaught Place, New Delhi",4.0,"N-55/56 & 88/89, Outer Circle, Connaught Place...",1179,4.035804,New Delhi,2,1.764706
649,Swagath,"[North Indian, Chinese, South Indian, Seafood]",2000,"Defence Colony Main Market, Defence Colony, N...",4.2,"Shop 13 - 14, Defence Colony Market, Near Defe...",649,3.562043,New Delhi,2,1.960784
1211,Singla's,"[North Indian, Mithai, Chinese, Fast Food, Str...",500,"IP Extension, New Delhi",4.0,"A-139/140, Main Road, Madhu Vihar, Near Hasanp...",1211,3.491553,New Delhi,2,0.490196
21,Olive Bar & Kitchen,"[Italian, Pizza, Mediterranean, Continental, E...",4000,"Mehrauli, New Delhi",4.7,"6-8, Kalka Das Marg, Mehrauli, New Delhi",21,4.506078,New Delhi,2,3.921569
